In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sets the theme of the charts
plt.style.use('seaborn-v0_8-darkgrid')

%matplotlib inline

In [ ]:
# imports the csv files
appointments_df = pd.read_csv('appointments.csv', low_memory=False, encoding='unicode_escape')

In [ ]:
# checks the content of the csv files
appointments_df.info()

### `pxid` variable

In [ ]:
# check values of pxid
appointments_df['pxid']

In [ ]:
# check unique values of pxid
appointments_df['pxid'].unique()

In [ ]:
# check value counts of pxid
appointments_df['pxid'].value_counts()

In [ ]:
# check for null values of pxid
pxid_count = appointments_df['pxid'].isnull().sum()

### `clinicid` variable

In [ ]:
# check values of clinicid
appointments_df['clinicid']

In [ ]:
# check unique values of clinicid
appointments_df['clinicid'].unique()

In [ ]:
# check value counts of clinicid
appointments_df['clinicid'].value_counts()

In [ ]:
# check for null values of clinicid
appointments_df['clinicid'].isnull().sum()

### `doctorid` variable

In [ ]:
# check values of doctorid
appointments_df['doctorid']

In [ ]:
# check unique values of doctorid
appointments_df['doctorid'].unique()

In [ ]:
# check value counts of doctorid
appointments_df['doctorid'].value_counts()

In [ ]:
# check null values of doctorid
appointments_df['doctorid'].isnull().sum()

### `apptid` variable

In [ ]:
# check values of apptid
appointments_df['apptid']

In [ ]:
# check unique values of apptid
appointments_df['apptid'].unique()

In [ ]:
# check value counts of apptid
appointments_df['apptid'].value_counts()

In [ ]:
# check null values of apptid
appointments_df['apptid'].isnull().sum()

### `status` variable

In [ ]:
# check values of status
appointments_df['status']

In [ ]:
# check unique values of status
appointments_df['status'].unique()

In [ ]:
# check count for status since there are two values for 'Complete'
status_vc = appointments_df['status'].value_counts()
print(status_vc)

In [ ]:
# replace Completed with Complete
appointments_df['status'] = appointments_df['status'].replace({'Completed': 'Complete'}, regex=True)
status_vc = appointments_df['status'].value_counts()
print(status_vc)

In [ ]:
# check null values of status
appointments_df['status'].isnull().sum()

### `TimeQueued` variable

In [ ]:
# check values of TimeQueued
appointments_df['TimeQueued']

In [ ]:
# check unique values of TimeQueued
appointments_df['TimeQueued'].unique()

In [ ]:
# check value counts of TimeQueued
appointments_df['TimeQueued'].value_counts()

In [ ]:
# check null values of TimeQueued
appointments_df['TimeQueued'].isnull().sum()

### `QueueDate` variable

In [ ]:
# check values of QueueDate
appointments_df['QueueDate']

In [ ]:
# check unique values of QueueDate
appointments_df['QueueDate'].unique()

In [ ]:
# check value counts of QueueDate
appointments_df['QueueDate'].value_counts()

In [ ]:
import datetime

# Function to convert datetime to date, removing time
remove_time = lambda s: datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S').date() \
                         if isinstance(s, str) and s.count(' ') >  0 \
                         else datetime.datetime.strptime(s, '%Y-%m-%d').date() if isinstance(s, str) \
                         else None  # or another value to represent a missing date

# Apply the function to the 'QueueDate' column
appointments_df['QueueDate'] = appointments_df['QueueDate'].apply(remove_time)

print(appointments_df['QueueDate'])

In [ ]:
# check null values of QueueDate
appointments_df['QueueDate'].isnull().sum()

### `StartTime` variable

In [ ]:
# check values of StartTime
appointments_df['StartTime']

In [ ]:
# check unique values of StartTime
appointments_df['StartTime'].unique()

In [ ]:
# check value counts of StartTime
appointments_df['StartTime'].value_counts()

In [ ]:
# check null values of StartTime
appointments_df['StartTime'].isnull().sum()

### `EndTime` variable

In [ ]:
# check values of EndTime
appointments_df['EndTime']

In [ ]:
# check unique values of EndTime
appointments_df['EndTime'].unique()

In [ ]:
# check value counts of EndTime
appointments_df['EndTime'].value_counts()

In [ ]:
# check null values of EndTime
appointments_df['EndTime'].isnull().sum()

### `type` variable

In [ ]:
# check values of type
appointments_df['type']

In [ ]:
# check unique values of type
appointments_df['type'].unique()

In [ ]:
type_vc = appointments_df['type'].value_counts()
print(type_vc)

In [ ]:
# check null values of type
appointments_df['type'].isnull().sum()

### `Virtual` variable

In [ ]:
# check values of Virtual
appointments_df['Virtual']

In [ ]:
# check unique values of Virtual
appointments_df['Virtual'].unique()

In [ ]:
virtual_vc = appointments_df['Virtual'].value_counts()
print(virtual_vc)

In [ ]:
# check null values of Virtual
appointments_df['Virtual'].isnull().sum()

### Exporting Files

In [ ]:
# Select the column you want to export
selected_column = appointments_df['QueueDate']

# Write the selected column to a new CSV file
selected_column.to_csv('newfile2.csv', index=False)